# 전문가를 위한 Subclassing API

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

[MNIST 데이터셋](http://yann.lecun.com/exdb/mnist/)을 로드하여 준비합니다.

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

11501568/11490434 [==============================] - 0s 0us/step


tf.data를 사용하여 데이터셋을 섞고 배치를 만듭니다:

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.batch(32)

케라스(Keras)의 [모델 서브클래싱(subclassing) API](https://www.tensorflow.org/guide/keras#model_subclassing)를 사용하여 `tf.keras` 모델을 만듭니다:

In [4]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# model instance 생성
model = MyModel()

훈련에 필요한 옵티마이저(optimizer)와 손실 함수를 선택합니다: 

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

모델의 손실과 성능을 측정할 지표를 선택합니다. 에포크가 진행되는 동안 수집된 측정 지표를 바탕으로 최종 결과를 출력합니다.

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

`tf.GradientTape`를 사용하여 모델을 훈련합니다:   

- with tf.GradientTape() as tape: 로 저장할 tape을 지정해주면, 이후의 GradientTape() 문맥 아래에서의 TensorFlow의 연관 연산 코드는 tape에 저장이 됩니다. 이렇게 tape에 저장된 연산 과정 (함수, 연산식) 을 가져다가 TensorFlow는 dx = tape.gradient(loss, x) 로 후진 모드 자동 미분 (Reverse mode automatic differentiation) 방법으로 손실에 대한 x의 미분을 계산합니다. 이렇게 계산한 손실에 대한 x의 미분을 역전파(backpropagation)하여 x의 값을 갱신(update)하는 작업을 반복하므로써 변수 x의 답을 찾아가는 학습을 진행합니다.   


- def 위에 @tf.fucnction annotation을 붙이면 마치 tf2.x 버전에서도 tf1.x 형태처럼 그래프 생성과 실행이 분리된 형태로 해당 함수내의 로직이 실행되게 됩니다. (실행 속도가 약간 빨라질 수 있습니다.)

In [7]:
@tf.function
def train_step(images, labels):
    
    with tf.GradientTape() as tape:
        # training=True는 학습과 추론시 다른 동작을 하는 
        # 레이어(예: Dropout)가 있는 경우에만 필요합니다.
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

모델을 테스트합니다:

In [8]:
@tf.function
def test_step(images, labels):
    # training=False는 학습과 추론시 다른 동작을 하는 
    # 레이어(예: Dropout)가 있는 경우에만 필요합니다.
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.14137952029705048, Accuracy: 95.75833129882812, Test Loss: 0.06441929191350937, Test Accuracy: 97.95999908447266
Epoch 2, Loss: 0.044733963906764984, Accuracy: 98.62333679199219, Test Loss: 0.05926070734858513, Test Accuracy: 98.11000061035156
Epoch 3, Loss: 0.022939037531614304, Accuracy: 99.28666687011719, Test Loss: 0.058805838227272034, Test Accuracy: 98.2699966430664
Epoch 4, Loss: 0.013840120285749435, Accuracy: 99.51666259765625, Test Loss: 0.05807633325457573, Test Accuracy: 98.2699966430664
Epoch 5, Loss: 0.009230741299688816, Accuracy: 99.6883316040039, Test Loss: 0.06031099706888199, Test Accuracy: 98.3499984741211


훈련된 이미지 분류기는 이 데이터셋에서 약 98%의 정확도를 달성합니다. 더 자세한 내용은 [TensorFlow 튜토리얼](https://www.tensorflow.org/tutorials/)을 참고하세요.